<a href="https://colab.research.google.com/github/olaviinha/NeuralStyleTransfer/blob/master/NeuralStyleTransfer_crowsonkb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Neural Style Transfer <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">style-transfer-pytorch</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralStyleTransfer" target="_blank"><font color="#999" size="4">Github</font></a>

Colab and batch prosessor for a [modified fork](https://github.com/olaviinha/style-transfer-pytorch) of [style-transfer-pytorch](https://github.com/crowsonkb/style-transfer-pytorch) by Katherine Crowson, an implementation of [A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576). This notebook takes **directories** of content images and style images, and styles each content image with each style image, saving them to output directory. Enter all directory paths relative to your Google Drive root.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

force_setup = False
pip_packages = 'typer rich gmic'

import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!git clone https://github.com/olaviinha/style-transfer-pytorch
!pip install -e ./style-transfer-pytorch

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@title # Style transfer
content_images_dir = "ai/st4/in" #@param {type:"string"}
style_images_dir = "ai/st4/style" #@param {type:"string"}
output_dir = "ai/st4/out" #@param {type:"string"}
min_scale = 512 #@param {type:"slider", min:512, max:2400, step:64}
max_scale = 2240 #@param {type:"slider", min:512, max:2400, step:64}
style_scale_factor = 1 #@param {type:"slider", min:0.01, max:10, step:0.01}
iterations = 500 #@param {type:"slider", min:10, max:1000, step:10}
# #@markdown <small>Every nth iteration is saved as separate file. Set to `0` to save only final output.</small>
# save_every = 0 #@param {type:"slider", min:0, max:500, step:100}
# #@markdown <small>Timestamp output files.</small>
# timestamp = False #@param {type:"boolean"}
save_every = 500
timestamp = False

remove_iterations = False
if save_every == 0:
  save_every = 500
  remove_iterations = True

content_images_dir = fix_path(drive_root+content_images_dir)
style_images_dir = fix_path(drive_root+style_images_dir)
output_dir = fix_path(drive_root+output_dir)

image_min = min_scale
image_max = max_scale

import os, random, string, ntpath
from os import listdir
from os.path import isfile, join
from glob import glob
import IPython
import requests

content_imgs = [f for f in listdir(content_images_dir) if isfile(join(content_images_dir, f))]
style_imgs = [f for f in listdir(style_images_dir) if isfile(join(style_images_dir, f))]

content_imgs.sort()
style_imgs.sort()

if timestamp is True:
  uniq_id = datetime.today().strftime('%Y%m%d%H%M%S')
else:
  uniq_id = requests.get('https://api.inha.asia/k/?type=short').text

%cd "{output_dir}"
for content_img in content_imgs:
  content_img_name = content_img
  content_img = content_images_dir+content_img
  for style_img in style_imgs:
    op(c.title, 'Styling '+content_img_name+' with '+style_img)
    style_img = style_images_dir+style_img
    output_img = output_dir + "NST_" + slug(basename(content_img)) + '_' + slug(basename(style_img)) + '__' + uniq_id + '.png'
    !style_transfer "{content_img}" "{style_img}" --output "{output_img}" --save-every {save_every} --min-scale {image_min} --end-scale {image_max} --iterations {iterations}
    #IPython.display.Image(output_img)

if remove_iterations is True:
  output_imgs = [f for f in listdir(output_dir) if isfile(join(output_dir, f))]
  for output_img in output_imgs:
    if '_iter_' in output_img:
      !rm "{output_dir}{output_img}"
      
op(c.ok, 'FIN.')